In [1]:
import pickle
import pandas as pd
import os
import torch

import betfairlightweight
from betfairlightweight import filters
from datetime import datetime
from datetime import timedelta
from dateutil import tz
import math
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.linear_model import LogisticRegression
import fasttrack as ft
import importlib
import featurecreations
from tqdm.notebook import tqdm
from sklearn.preprocessing import OneHotEncoder


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [14]:
# Standard locations
betfair_sp_file = (
    r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\df-betfairSP.npy"
)
previous_results_file = (
    r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\all-results-db.npy"
)
split_dist_file = (
    r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\split_dist_ft.csv"
)
prev_full_details = r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\full_race_details_ft.csv"


In [20]:
importlib.reload(ft)
today = datetime.today().strftime('%Y-%m-%d')
seckey = "50efd775-e988-4be3-924f-87631fabdc3f"
greys = ft.Fasttrack(seckey)
race_details, dog_results = greys.getRaceResults('2022-12-01', dt_end=today)

Valid Security Key
Getting meets for each date ..


100%|██████████| 43/43 [00:23<00:00,  1.83it/s]


Getting historic results details ..


100%|██████████| 562/562 [06:03<00:00,  1.55it/s]


In [21]:
importlib.reload(featurecreations)
x = featurecreations.generate_results_df(previous_results_file, dog_results, race_details, betfair_sp_file, split_dist_file)

c:\Users\Nick\Documents\GitHub\grvmodel\Python\Model Prediction\featurecreations.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race_details['RaceId'] = race_details['@id']


(51110, 19)
(42915, 19)


100%|██████████| 155095/155095 [52:58<00:00, 48.80it/s]  


In [23]:
len(x.trackOHE.iloc[0])

74

In [24]:
with open("new gru input 2023-01.npy", "wb") as fp:   #Pickling
    
    pickle.dump(x, fp)

In [64]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
race_ft_details = pd.DataFrame(pickle.load(open('race_details_ft.npy','rb')), columns=['@id', 'RaceNum', 'RaceName','RaceTime', 'Distance', 'RaceGrade', 'Track', 'date'])
race_ft_details['RaceId'] = race_ft_details['@id'] 
dog_ft_details = pd.DataFrame(pickle.load(open('dog_details_ft.npy','rb')))
dog_ft_details['DogId'] =   dog_ft_details['@id']
full_details = pd.merge(race_ft_details,dog_ft_details, 'right', on='RaceId')